In [67]:
import os
import rioxarray as rxr
import xarray as xr
from dataclasses import asdict
from dscreator.cfarray.attributes import DatasetAttrsGrid, VariableAttrs
from datetime import datetime
from dscreator import utils
from dscreator.storage import BaseHandler, get_storage_handler
from dscreator.sources.ferrybox.extractor import TrajectoryExtractor
from sqlalchemy import create_engine
from dscreator.config import SETTINGS
from dscreator.datasets.trajectories.ferrybox import FerryboxTrajBuilder

In [68]:
boat = "FA"
measurement_parameters = ["Temperature", "Salinity", "Oxygen"]
uuid="29b7de62-e1fa-4dce-90e4-7ff8a0931397"
datasetname="FA_ferrybox_2017_2022"
stationname="Color Fantasy"
projectname="NorSoop"

In [69]:
engine = create_engine(SETTINGS.database_url)
te = TrajectoryExtractor(engine, boat, measurement_parameters)
ts = te.fetch_slice(start_time=datetime(2021, 12, 12, 16, 0, 0), end_time=datetime(2021, 12, 13, 16, 0, 0))
print(f"Fetched data ts")

Fetched data ts


In [70]:
print(f"Manual qc is not yet implemented")
# print(f"Data after manual qc {ts}")

Manual qc is not yet implemented


In [71]:
tb = FerryboxTrajBuilder(
    uuid=uuid,
    dataset_name=datasetname,
    station_name=stationname,
    project_name=projectname,
    is_acdd=True,
)
ds = tb.create(ts)
print(f"ds {ds}")

ds <xarray.Dataset>
Dimensions:                (time: 965)
Coordinates:
  * time                   (time) datetime64[ns] 2021-12-12T16:00:12 ... 2021...
    longitude              (time) float64 10.92 10.93 10.93 ... 10.64 10.64
    latitude               (time) float64 54.95 54.96 54.96 ... 59.12 59.11
Data variables:
    sea_water_temperature  (time) float64 6.468 6.456 6.459 ... 5.403 5.245 5.22
    salinity               (time) float64 9.928 9.906 9.857 ... 5.671 5.687 5.69
    oxygen                 (time) float64 93.88 95.03 93.84 ... 94.12 94.74
    trajectory_name        <U13 'Color Fantasy'
Attributes: (12/36)
    featureType:             trajectory
    id:                      29b7de62-e1fa-4dce-90e4-7ff8a0931397
    title:                   Ferrybox on MS Color Fantasy
    summary:                 Ferry sailing from Oslo to Kiel. For more inform...
    date_created:            2023-04-13 11:59:04.191244
    keywords:                
    ...                      ...
    publi

In [73]:
if ds.dims["time"] > 0:
    print(f"Saving dataset slice {ds.time[0].values} --> {ds.time[-1].values}")

    sh= get_storage_handler(
        project_name=str(datasetname),
        dataset_name=str(stationname),
        unlimited_dims=["time"],
        filename_prefix= "FA"
    )
    print(f"sh {sh}")
    print(ds)
    sh.save_dataset(ds)
else:
    print("Found no data for interval")

Saving dataset slice 2021-12-12T16:00:12.000000000 --> 2021-12-13T15:59:47.000000000
sh LocalStorageHandler(project_name='FA_ferrybox_2017_2022', dataset_name='Color Fantasy', filename_prefix='FA', workdir='datasets/fa_ferrybox_2017_2022/Color Fantasy', restart_filepath='datasets/fa_ferrybox_2017_2022/Color Fantasy/.Color Fantasy.restart.json', unlimited_dims=['time'])
<xarray.Dataset>
Dimensions:                (time: 965)
Coordinates:
  * time                   (time) datetime64[ns] 2021-12-12T16:00:12 ... 2021...
    longitude              (time) float64 10.92 10.93 10.93 ... 10.64 10.64
    latitude               (time) float64 54.95 54.96 54.96 ... 59.12 59.11
Data variables:
    sea_water_temperature  (time) float64 6.468 6.456 6.459 ... 5.403 5.245 5.22
    salinity               (time) float64 9.928 9.906 9.857 ... 5.671 5.687 5.69
    oxygen                 (time) float64 93.88 95.03 93.84 ... 94.12 94.74
    trajectory_name        <U13 'Color Fantasy'
Attributes: (12/36)
    

ValueError: unexpected encoding for scipy backend: ['zlib', '_FillValue']